# OpenET Data Scraper
Scrapes OpenET data from the ClimateEngine API for all input subcatchments. 

## 1. Setup

#### Environment
To run this notebook, Python and some python dependencies are required. For convenience, a `requirements.txt` will be provided.

#### API Key
Define an environment variable `CLIMATE_ENGINE_API_KEY` with your climate engine API key. If you do not have a key, [request a key](https://support.climateengine.org/article/36-requesting-an-authorization-key-token).

In [152]:
import os
from dotenv import load_dotenv

load_dotenv()
CLIMATE_ENGINE_API_KEY = os.environ.get('CLIMATE_ENGINE_API_KEY')

HEADERS = {
    'Accept': 'application/json',
    'Authorization': CLIMATE_ENGINE_API_KEY
}

#### Subcatchments Manifest
Provide a manifest for the locations of all the relevant subcatchments and their geometries. This script currently supports `.yaml` manifests with the following structure:
```
Subcatchments:
  - name:
    url:
  - name:
    url:
  ...
```
where the urls points to geojson with subcatchments as features. From each data file, the topaz id and geometry will be used to create the final output.
      

In [153]:
SUBCATCHMENTS_MANIFEST = 'subcatchment-manifest.yaml'

#### Misc. Configuration
Some configuration options will be available below for tweaking/debugging.

In [154]:
VERBOSE = True # Detailed logging
API_BASE_URL = 'https://api.climateengine.org'

## 2. Helper Functions
The functions used will be defined below in a way that explains the implementation for building the output of one watershed. For a simpler view of the entire process, see the [main loop](#main-loop).

#### Retry Logic

The requests made in this project occasionally fail, but can be fixed with a simple retry. Thus, requests will be made with retry-enabled aiohttp requests. The aiohttp library is used over the requests library for compatibility with asyncio. 


In [155]:
import aiohttp
from tenacity import (
    retry,
    stop_after_attempt,
    wait_exponential,
    retry_if_exception_type,
    retry_if_result
)

def is_server_error(response):
    """Check if response status indicates a server error."""
    return response is not None and response.status >= 500

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=1, max=10),
    retry=(
        retry_if_exception_type((aiohttp.ClientError, TimeoutError)) |
        retry_if_result(is_server_error)
    ),
    reraise=True  # Re-raise so we can catch and handle at call site
)
async def fetch_with_retry(session: aiohttp.ClientSession, method: str, url: str, **kwargs):
    """
    Fetch a URL with automatic retry logic using tenacity.
    
    Parameters:
    - session: aiohttp ClientSession
    - method: HTTP method ('GET', 'POST', etc.)
    - url: URL to fetch
    - **kwargs: Additional arguments to pass to session.request()
    
    Returns:
    - aiohttp.ClientResponse object
    
    Raises:
    - Exception if all retries are exhausted
    
    Retries on:
    - Server errors (5xx status codes)
    - Network errors (ClientError, TimeoutError)
    - Uses exponential backoff (1s, 2s, 4s, 8s, max 10s)
    """
    response = await session.request(method, url, **kwargs)
    
    # If server error, trigger retry
    if response.status >= 500:
        await response.release()
        return response  # Will trigger retry via retry_if_result
    
    return response


#### Prepare Subcatchments
The subcatchment topaz ids and geometries need to be collected - this function fetches them for a single watershed and returns them in a Pandas dataframe. Note that multiple geometries can exist for the same topazid.

In [156]:
import pandas as pd
import logging

logger = logging.getLogger()

async def get_subcatchments(geojson_url: str, verbose: bool = False) -> pd.DataFrame:
    """
    Fetches the geojson data from a URL and returns a pandas DataFrame of subcatchment geometries.
    """
    if verbose:
        logging.basicConfig(level=logging.INFO)
        logger.setLevel(logging.INFO)

    async with aiohttp.ClientSession() as session:
        try:
            response = await fetch_with_retry(session, 'GET', geojson_url)
            
            if response.status == 200:
                geojson_data = await response.json()
                logger.info(f"Loaded {len(geojson_data['features'])} subcatchment features (geometries)")
                
                # Extract unique TopazIDs and their geometries
                subcatchments = []
                for feature in geojson_data['features']:
                    topaz_id = feature['properties']['TopazID']
                    geometry = feature['geometry']
                    subcatchments.append({'topaz_id': topaz_id, 'geometry': geometry})
                
                # Create DataFrame and get unique subcatchments
                subcatchments_df = pd.DataFrame(subcatchments)
                unique_subcatchments = subcatchments_df.groupby('topaz_id').first().reset_index()
                logger.info(f"Found {len(unique_subcatchments)} unique TopazIDs")
                logger.info(f"Sample TopazIDs: {unique_subcatchments['topaz_id'].head().tolist()}")

                return subcatchments_df
            else:
                text = await response.text()
                logger.error(f'Error {response.status}: {text}')
                return pd.DataFrame()
        except aiohttp.ClientError as e:
            logger.error(f'Network error fetching subcatchments (after retries): {type(e).__name__}: {str(e)}')
            return pd.DataFrame()
        except Exception as e:

            logger.error(f'Failed to fetch subcatchments (after retries): {type(e).__name__}: {str(e)}')
            return pd.DataFrame()

#### Fetch OpenET Timeseries

The ClimateEngine API accepts geometries and can return ET timeseries data. For more information, see:
* https://docs.climateengine.org/docs/build/html/timeseries.html#rst-timeseries-native-coordinates
* https://support.climateengine.org/article/152-formatting-coordinates-for-api-requests

The following functions concurrently fetch ET timeseries data from the API for each subcatchment in a watershed. The concurrency is implemented using asyncio. 

Single request:

In [157]:
import asyncio
import json
from urllib.parse import urlencode

async def fetch_single_et_timeseries(
    session: aiohttp.ClientSession,
    geometry: dict,
    dataset: str,
    variable: str,
    start_date: str,
    end_date: str,
    area_reducer: str = 'mean'
) -> dict:
    """
    Fetch ET timeseries data for a polygon geometry.
    
    Parameters:
    - session: aiohttp ClientSession (should be reused across multiple calls)
    - geometry: GeoJSON geometry object (Polygon)
    - dataset: Dataset name (e.g., 'OpenET_ENSEMBLE')
    - variable: ET variable name (e.g., 'et_ensemble_mad')
    - start_date: Start date (YYYY-MM-DD format)
    - end_date: End date (YYYY-MM-DD format)
    - area_reducer: Spatial aggregation method (default: 'mean')
    
    Returns:
    - API response with timeseries data, or None if request fails
    
    Note: 
        - NoData values (-9999.0) are NOT filtered here
        - Uses automatic retry with exponential backoff via tenacity.
        - Coordinates must be formatted as per ClimateEngine API spec:
          For polygons: [[[-121.61,38.78],[-121.52,38.78],[-121.52,38.83],[-121.61,38.83],[-121.61,38.78]]]
    """
    url = f'{API_BASE_URL}/timeseries/native/coordinates'
    
    # Prepare query parameters with coordinates as JSON string
    params = {
        'dataset': dataset,
        'variable': variable,
        'start_date': start_date,
        'end_date': end_date,
        'area_reducer': area_reducer,
        'coordinates': json.dumps(geometry['coordinates'])
    }
    
    try:
        response = await fetch_with_retry(session, 'GET', url, headers=HEADERS, params=params)
        
        if response.status == 200:
            data = await response.json()
            return data
        else:
            text = await response.text()
            logger.error(f'ET API request failed with status {response.status}: {text[:200]}')
            return None
            
    except aiohttp.ClientError as e:
        logger.error(f'Network error during ET API request (after retries): {type(e).__name__}: {str(e)}')
        return None
    except Exception as e:
        logger.error(f'Exception during ET API request (after retries): {type(e).__name__}: {str(e)}')
        return None


Requests for entire watershed (concurrent):

In [158]:
import time
from tqdm.asyncio import tqdm

async def fetch_all_et_data(
    subcatchments_df: pd.DataFrame,
    dataset: str = 'OpenET_ENSEMBLE',
    variable: str = 'et_ensemble_mad',
    start_date: str = '2023-01-01',
    end_date: str = '2023-12-31',
    area_reducer: str = 'mean',
    max_concurrent: int = 1000
) -> tuple[pd.DataFrame, dict]:
    """
    Fetch ET timeseries data concurrently for all subcatchments with progress bar.
    
    Parameters:
    - subcatchments_df: DataFrame with 'topaz_id' and 'geometry' columns
    - dataset: Dataset name (default: 'OpenET_ENSEMBLE')
    - variable: ET variable name (default: 'et_ensemble_mad')
    - start_date: Start date (YYYY-MM-DD)
    - end_date: End date (YYYY-MM-DD)
    - area_reducer: Spatial aggregation method (default: 'mean')
    - max_concurrent: Maximum concurrent requests (default: 50)
    
    Returns:
    - Tuple of (DataFrame with topaz_id, date, and ET value columns, fetch statistics dict)
    """
    results = []
    start_time = time.time()
    
    # Track success/failure statistics
    successful_fetches = 0
    failed_fetches = 0
    failed_topaz_ids = []
    
    async with aiohttp.ClientSession() as session:
        # Use asyncio.Semaphore to limit concurrent requests
        semaphore = asyncio.Semaphore(max_concurrent)
        
        # Create async wrapper for each fetch with semaphore
        async def fetch_with_semaphore(topaz_id, geometry):
            async with semaphore:
                response = await fetch_single_et_timeseries(
                    session=session,
                    geometry=geometry,
                    dataset=dataset,
                    variable=variable,
                    start_date=start_date,
                    end_date=end_date,
                    area_reducer=area_reducer
                )
                return topaz_id, response
        
        # Create task objects (not coroutines) for all subcatchments
        tasks = [
            asyncio.create_task(fetch_with_semaphore(row['topaz_id'], row['geometry']))
            for _, row in subcatchments_df.iterrows()
        ]
        
        print(f"Starting concurrent fetch for {len(tasks)} subcatchments...")
        print(f"Max concurrent requests: {max_concurrent}")
        
        # Use tqdm.asyncio.gather for progress bar with actual task objects
        responses = []
        for coro in tqdm.as_completed(tasks, total=len(tasks), desc="Fetching ET data"):
            result = await coro
            responses.append(result)
        
        # Process results
        for topaz_id, response in responses:
            if isinstance(response, Exception):
                logger.error(f"Failed to fetch data for TopazID {topaz_id}: {str(response)}")
                failed_fetches += 1
                failed_topaz_ids.append(topaz_id)
                continue
            
            if response and "Data" in response:
                successful_fetches += 1
                for entry in response["Data"][0]["Data"]:
                    date = entry.get("Date")
                    value = entry.get(f"{variable} (mm)")
                    results.append({
                        "topaz_id": topaz_id,
                        "date": date,
                        "et_value": value
                    })
            else:
                failed_fetches += 1
                failed_topaz_ids.append(topaz_id)
                if response is None:
                    logger.warning(f"No response returned for TopazID {topaz_id}")
                else:
                    logger.warning(f"No data in response for TopazID {topaz_id}")
    
    elapsed_time = time.time() - start_time
    total_requests = len(tasks)
    success_rate = (successful_fetches / total_requests * 100) if total_requests > 0 else 0
    
    print(f"\nCompleted in {elapsed_time:.2f} seconds ({elapsed_time/len(tasks):.2f}s per subcatchment)")
    print(f"Success: {successful_fetches}/{total_requests} ({success_rate:.1f}%)")
    
    if failed_fetches > 0:
        print(f"⚠ Failed: {failed_fetches} subcatchments")
        if len(failed_topaz_ids) <= 10:
            print(f"  Failed TopazIDs: {failed_topaz_ids}")
        else:
            print(f"  Failed TopazIDs: {failed_topaz_ids[:10]} ... and {len(failed_topaz_ids)-10} more")
    
    # Compile statistics - convert to native Python types for JSON serialization
    fetch_stats = {
        'total': int(total_requests),
        'successful': int(successful_fetches),
        'failed': int(failed_fetches),
        'success_rate': float(success_rate),
        'elapsed_time_seconds': float(elapsed_time),
        'failed_topaz_ids': failed_topaz_ids
    }
    
    return pd.DataFrame(results), fetch_stats

#### Handling Multipolygons

Subcatchments can take the form of a multipolygon (multiple disjointed polygons). This will manifest as multiple geometries associated with the same topazid in the table returned by the last step. There are several ways to handle this:
* weighted average
* drop other polygons and keep the largest

#### Data Quality: Missing Values

The ClimateEngine API may return `-9999` (or `-9999.0`) to indicate missing data. For data integrity, we need to identify and filter out subcatchments that contain any missing values.

**Approach:**
- Identify all subcatchments (topaz_ids) that have at least one `-9999` value
- Filter out these subcatchments completely from the dataset
- This is done regardless of whether the subcatchment is a multipolygon with some valid values

In [159]:
def identify_subcatchments_with_missing_values(et_data_df: pd.DataFrame, missing_value: float = -9999.0) -> set:
    """
    Identify subcatchments that have any missing values in their ET data.
    
    Parameters:
    - et_data_df: DataFrame with 'topaz_id' and 'et_value' columns
    - missing_value: The value that indicates missing data (default: -9999.0)
    
    Returns:
    - Set of topaz_ids that contain at least one missing value
    """
    # Find all rows with missing values
    missing_mask = et_data_df['et_value'] == missing_value
    
    # Get unique topaz_ids that have missing values
    topaz_ids_with_missing = set(et_data_df[missing_mask]['topaz_id'].unique())
    
    return topaz_ids_with_missing


def filter_out_missing_values(et_data_df: pd.DataFrame, subcatchments_df: pd.DataFrame, missing_value: float = -9999.0) -> tuple[pd.DataFrame, pd.DataFrame, dict]:
    """
    Filter out subcatchments that have any missing values from both the ET data and subcatchments DataFrames.
    
    Parameters:
    - et_data_df: DataFrame with 'topaz_id' and 'et_value' columns
    - subcatchments_df: DataFrame with 'topaz_id' and 'geometry' columns
    - missing_value: The value that indicates missing data (default: -9999.0)
    
    Returns:
    - Tuple of (filtered_et_data_df, filtered_subcatchments_df, filter_stats)
    """
    # Identify subcatchments with missing values
    topaz_ids_with_missing = identify_subcatchments_with_missing_values(et_data_df, missing_value)
    
    # Count statistics
    total_subcatchments = et_data_df['topaz_id'].nunique()
    num_with_missing = len(topaz_ids_with_missing)
    num_retained = total_subcatchments - num_with_missing
    
    # Filter both dataframes
    filtered_et_data = et_data_df[~et_data_df['topaz_id'].isin(topaz_ids_with_missing)].copy()
    filtered_subcatchments = subcatchments_df[~subcatchments_df['topaz_id'].isin(topaz_ids_with_missing)].copy()
    
    # Calculate record statistics
    total_records = len(et_data_df)
    records_removed = len(et_data_df) - len(filtered_et_data)
    
    filter_stats = {
        'total_subcatchments': int(total_subcatchments),
        'subcatchments_with_missing': int(num_with_missing),
        'subcatchments_retained': int(num_retained),
        'total_records': int(total_records),
        'records_removed': int(records_removed),
        'records_retained': int(len(filtered_et_data))
    }
    
    return filtered_et_data, filtered_subcatchments, filter_stats

#### Multipolygon Aggregation

Some subcatchments consist of multiple disjointed polygons (multipolygons) represented by multiple geometries for the same topaz_id. We need to aggregate the ET data from these multiple polygons into a single value per time step.

**Two aggregation strategies:**

1. **Area-weighted aggregation**: Calculate a weighted average based on the area of each polygon
   - More accurate representation of the entire subcatchment
   - Preserves information from all polygons
   - Requires calculating polygon areas

2. **Largest polygon only**: Use the ET value from the largest polygon
   - Simpler and faster
   - May lose information from smaller polygons
   - Assumes the largest polygon is most representative

In [160]:
from shapely.geometry import shape
from shapely.ops import transform
import pyproj


def calculate_polygon_area(geometry: dict) -> float:
    """
    Calculate the area of a polygon in square meters.
    
    Parameters:
    - geometry: GeoJSON geometry object
    
    Returns:
    - Area in square meters
    """
    # Convert GeoJSON to shapely geometry
    geom = shape(geometry)
    
    # Get the centroid to determine appropriate UTM zone
    centroid = geom.centroid
    lon, lat = centroid.x, centroid.y
    
    # Calculate UTM zone
    utm_zone = int((lon + 180) / 6) + 1
    
    # Create transformer from WGS84 to UTM using modern API
    wgs84 = pyproj.CRS("EPSG:4326")
    utm = pyproj.CRS(f"+proj=utm +zone={utm_zone} +datum=WGS84 +units=m +no_defs")
    transformer = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True)
    
    # Transform and calculate area
    geom_utm = transform(transformer.transform, geom)
    return geom_utm.area


def identify_multipolygons(subcatchments_df: pd.DataFrame) -> dict:
    """
    Identify which topaz_ids have multiple geometries (multipolygons).
    
    Parameters:
    - subcatchments_df: DataFrame with 'topaz_id' and 'geometry' columns
    
    Returns:
    - Dictionary with statistics about multipolygons
    """
    # Count geometries per topaz_id
    geometry_counts = subcatchments_df.groupby('topaz_id').size()
    
    # Identify multipolygons (more than 1 geometry)
    multipolygon_ids = geometry_counts[geometry_counts > 1].index.tolist()
    
    stats = {
        'total_unique_topaz_ids': int(subcatchments_df['topaz_id'].nunique()),
        'multipolygon_topaz_ids': int(len(multipolygon_ids)),
        'single_polygon_topaz_ids': int(len(geometry_counts[geometry_counts == 1])),
        'max_geometries_per_topaz_id': int(geometry_counts.max()),
        'multipolygon_ids': multipolygon_ids
    }
    
    return stats


def aggregate_multipolygons_weighted(
    et_data_df: pd.DataFrame,
    subcatchments_df: pd.DataFrame
) -> tuple[pd.DataFrame, dict]:
    """
    Aggregate multipolygon ET data using area-weighted averaging.
    
    For each topaz_id with multiple geometries:
    1. Calculate the area of each polygon
    2. For each time step, compute weighted average: sum(value_i * area_i) / sum(area_i)
    
    Parameters:
    - et_data_df: DataFrame with 'topaz_id', 'date', and 'et_value' columns
    - subcatchments_df: DataFrame with 'topaz_id' and 'geometry' columns
    
    Returns:
    - Tuple of (aggregated_et_data_df, aggregation_stats)
    """
    # Calculate areas for all geometries
    print("Calculating polygon areas...")
    subcatchments_df = subcatchments_df.copy()
    subcatchments_df['area'] = subcatchments_df['geometry'].apply(calculate_polygon_area)
    
    # Create a mapping of (topaz_id, geometry_index) to area
    # We need to track which geometry each ET value corresponds to
    subcatchments_df['geometry_index'] = subcatchments_df.groupby('topaz_id').cumcount()
    
    # Merge ET data with geometry areas
    # First, we need to add geometry_index to et_data
    # For this, we'll merge with subcatchments on topaz_id and use geometry_index order
    
    # Create geometry_index in et_data based on the order from subcatchments
    et_data_with_geometry = et_data_df.copy()
    
    # For each topaz_id, assign geometry_index based on order in subcatchments_df
    geometry_lookup = subcatchments_df.set_index(['topaz_id', 'geometry_index'])['area'].to_dict()
    
    # Create a geometry_index for ET data by counting occurrences
    et_data_with_geometry['geometry_index'] = et_data_with_geometry.groupby('topaz_id').cumcount() % subcatchments_df.groupby('topaz_id').size().max()
    
    # Map areas to ET data
    et_data_with_geometry['area'] = et_data_with_geometry.apply(
        lambda row: geometry_lookup.get((row['topaz_id'], row['geometry_index']), 1.0),
        axis=1
    )
    
    # Calculate weighted values
    et_data_with_geometry['weighted_value'] = et_data_with_geometry['et_value'] * et_data_with_geometry['area']
    
    # Group by topaz_id and date, then calculate weighted average
    aggregated = et_data_with_geometry.groupby(['topaz_id', 'date']).agg({
        'weighted_value': 'sum',
        'area': 'sum',
        'et_value': 'count'  # Count number of polygons
    }).reset_index()
    
    aggregated['et_value'] = aggregated['weighted_value'] / aggregated['area']
    aggregated = aggregated.rename(columns={'et_value_count': 'num_polygons'})
    aggregated = aggregated[['topaz_id', 'date', 'et_value']]
    
    # Calculate statistics
    original_records = len(et_data_df)
    aggregated_records = len(aggregated)
    multipolygon_count = (et_data_df.groupby('topaz_id').size() > 1).sum()
    
    stats = {
        'method': 'area_weighted',
        'original_records': int(original_records),
        'aggregated_records': int(aggregated_records),
        'records_reduced': int(original_records - aggregated_records),
        'multipolygon_topaz_ids': int(multipolygon_count)
    }
    
    return aggregated, stats


def aggregate_multipolygons_largest(
    et_data_df: pd.DataFrame,
    subcatchments_df: pd.DataFrame
) -> tuple[pd.DataFrame, dict]:
    """
    Aggregate multipolygon ET data by selecting only the largest polygon.
    
    For each topaz_id with multiple geometries:
    1. Calculate the area of each polygon
    2. Keep only the ET data from the largest polygon
    
    Parameters:
    - et_data_df: DataFrame with 'topaz_id', 'date', and 'et_value' columns
    - subcatchments_df: DataFrame with 'topaz_id' and 'geometry' columns
    
    Returns:
    - Tuple of (filtered_et_data_df, aggregation_stats)
    """
    # Calculate areas for all geometries
    print("Calculating polygon areas...")
    subcatchments_df = subcatchments_df.copy()
    subcatchments_df['area'] = subcatchments_df['geometry'].apply(calculate_polygon_area)
    
    # For each topaz_id, find the index of the largest polygon
    subcatchments_df['geometry_index'] = subcatchments_df.groupby('topaz_id').cumcount()
    
    # Find the geometry_index with max area for each topaz_id
    largest_geometries = subcatchments_df.loc[
        subcatchments_df.groupby('topaz_id')['area'].idxmax()
    ][['topaz_id', 'geometry_index']].set_index('topaz_id')['geometry_index'].to_dict()
    
    # Add geometry_index to ET data
    et_data_with_geometry = et_data_df.copy()
    et_data_with_geometry['geometry_index'] = et_data_with_geometry.groupby('topaz_id').cumcount() % subcatchments_df.groupby('topaz_id').size().max()
    
    # Filter to keep only data from largest polygons
    et_data_with_geometry['is_largest'] = et_data_with_geometry.apply(
        lambda row: row['geometry_index'] == largest_geometries.get(row['topaz_id'], 0),
        axis=1
    )
    
    filtered_data = et_data_with_geometry[et_data_with_geometry['is_largest']][['topaz_id', 'date', 'et_value']]
    
    # Calculate statistics
    original_records = len(et_data_df)
    filtered_records = len(filtered_data)
    multipolygon_count = len(largest_geometries)
    
    stats = {
        'method': 'largest_polygon',
        'original_records': int(original_records),
        'filtered_records': int(filtered_records),
        'records_removed': int(original_records - filtered_records),
        'multipolygon_topaz_ids': int(multipolygon_count)
    }
    
    return filtered_data, stats

#### Usage Notes

The `process_single_watershed` function now includes two optional parameters:

1. **`filter_missing` (bool, default=True)**: Automatically filters out subcatchments with any `-9999` values
   - Set to `False` to keep all data including subcatchments with missing values
   
2. **`multipolygon_method` (str, default='weighted')**: Chooses how to handle multipolygons
   - `'weighted'`: Area-weighted aggregation (more accurate)
   - `'largest'`: Use only the largest polygon (simpler, faster)
   
Processing statistics for both operations are saved in the metadata JSON file.

**Example usage:**

```python
# Option 1: Use defaults (filter missing values, weighted aggregation)
metadata = await process_single_watershed(
    watershed_name='my-watershed',
    geojson_url='https://example.com/data.geojson',
    dataset='OPENET_CONUS',
    variable='et_ensemble_mad',
    start_date='2020-01-01',
    end_date='2023-12-31'
)

# Option 2: Keep missing values, use largest polygon method
metadata = await process_single_watershed(
    watershed_name='my-watershed',
    geojson_url='https://example.com/data.geojson',
    dataset='OPENET_CONUS',
    variable='et_ensemble_mad',
    start_date='2020-01-01',
    end_date='2023-12-31',
    filter_missing=False,
    multipolygon_method='largest'
)
```

## Main Loop
Process watersheds from the manifest file.

In [161]:
async def process_single_watershed(
    watershed_name: str,
    geojson_url: str,
    dataset: str = 'OPENET_CONUS',
    variable: str = 'et_ensemble_mad',
    start_date: str = '2023-01-01',
    end_date: str = '2023-12-31',
    area_reducer: str = 'mean',
    max_concurrent: int = 10,
    output_dir: str = 'data',
    filter_missing: bool = True,
    multipolygon_method: str = 'weighted'
) -> dict:
    """
    Process all subcatchments for a single watershed and save results.
    
    Parameters:
    - watershed_name: Name of the watershed
    - geojson_url: URL to the watershed's GeoJSON file
    - dataset: Dataset name (default: 'OPENET_CONUS')
    - variable: ET variable name (default: 'et_ensemble_mad')
    - start_date: Start date (YYYY-MM-DD)
    - end_date: End date (YYYY-MM-DD)
    - area_reducer: Spatial aggregation method (default: 'mean')
    - max_concurrent: Maximum concurrent requests (default: 10)
    - output_dir: Directory to save output files (default: 'data')
    - filter_missing: Whether to filter out subcatchments with missing values (default: True)
    - multipolygon_method: Method for handling multipolygons - 'weighted' or 'largest' (default: 'weighted')
    
    Returns:
    - Dictionary with processing statistics
    """
    print(f"\n{'='*60}")
    print(f"Processing watershed: {watershed_name}")
    print(f"{'='*60}")
    
    # Load subcatchments
    logger.info(f"Loading subcatchments for {watershed_name}...")
    subcatchments_df = await get_subcatchments(geojson_url, verbose=VERBOSE)
    
    if subcatchments_df is None or len(subcatchments_df) == 0:
        print(f"⚠ Failed to load subcatchments for {watershed_name}")
        return None
    
    print(f"Loaded {len(subcatchments_df)} subcatchments")
    
    # Fetch ET data
    et_data_df, fetch_stats = await fetch_all_et_data(
        subcatchments_df=subcatchments_df,
        dataset=dataset,
        variable=variable,
        start_date=start_date,
        end_date=end_date,
        area_reducer=area_reducer,
        max_concurrent=max_concurrent
    )
    
    # Filter out missing values if requested
    missing_value_stats = None
    if filter_missing and len(et_data_df) > 0:
        print(f"\nFiltering subcatchments with missing values...")
        et_data_df, subcatchments_df, missing_value_stats = filter_out_missing_values(
            et_data_df, subcatchments_df, missing_value=-9999.0
        )
        print(f"  Removed {missing_value_stats['subcatchments_with_missing']} subcatchments with missing values")
        print(f"  Retained {missing_value_stats['subcatchments_retained']} subcatchments ({missing_value_stats['records_retained']} records)")
    
    # Handle multipolygons if requested
    multipolygon_stats = None
    if len(et_data_df) > 0:
        # First identify if there are multipolygons
        mp_info = identify_multipolygons(subcatchments_df)
        if mp_info['multipolygon_topaz_ids'] > 0:
            print(f"\nAggregating multipolygons using '{multipolygon_method}' method...")
            print(f"  Found {mp_info['multipolygon_topaz_ids']} multipolygons (out of {mp_info['total_unique_topaz_ids']} total)")
            
            if multipolygon_method == 'weighted':
                et_data_df, multipolygon_stats = aggregate_multipolygons_weighted(
                    et_data_df, subcatchments_df
                )
            elif multipolygon_method == 'largest':
                et_data_df, multipolygon_stats = aggregate_multipolygons_largest(
                    et_data_df, subcatchments_df
                )
            else:
                print(f"  ⚠ Unknown multipolygon_method '{multipolygon_method}', skipping aggregation")
            
            if multipolygon_stats:
                print(f"  Aggregated from {multipolygon_stats['original_records']} to {multipolygon_stats.get('aggregated_records', multipolygon_stats.get('filtered_records', 0))} records")
    
    # Save results
    os.makedirs(output_dir, exist_ok=True)
    
    # Sanitize watershed name for filename
    safe_name = watershed_name.replace(';;', '_').replace('/', '_')
    output_file = os.path.join(output_dir, f'et_data_{safe_name}.parquet')
    
    if len(et_data_df) > 0:
        # Add year and month columns for consistency with exploration notebook
        et_data_df['year'] = pd.to_datetime(et_data_df['date']).dt.year
        et_data_df['month'] = pd.to_datetime(et_data_df['date']).dt.month
        et_data_df['model'] = variable
        
        # Reorder columns: topaz_id, year, month, model, value (rename et_value to value)
        et_data_df = et_data_df.rename(columns={'et_value': 'value'})
        et_data_df = et_data_df[['topaz_id', 'year', 'month', 'model', 'value']]
        
        # Save to parquet
        et_data_df.to_parquet(output_file, index=False, engine='pyarrow', compression='snappy')
        file_size_mb = os.path.getsize(output_file) / (1024 * 1024)
    else:
        file_size_mb = 0
        output_file = None
        print(f"⚠ No data collected for {watershed_name}")
    
    # Create metadata
    metadata = {
        'watershed_name': watershed_name,
        'date_created': pd.Timestamp.now().isoformat(),
        'num_records': int(len(et_data_df)),
        'num_subcatchments': int(et_data_df['topaz_id'].nunique()) if len(et_data_df) > 0 else 0,
        'models': [variable],
        'date_range': {
            'start': start_date,
            'end': end_date
        },
        'years': [int(y) for y in sorted(et_data_df['year'].unique())] if len(et_data_df) > 0 else [],
        'dataset': dataset,
        'variable': variable,
        'area_reducer': area_reducer,
        'fetch_stats': fetch_stats,
        'filter_missing': filter_missing,
        'missing_value_stats': missing_value_stats,
        'multipolygon_method': multipolygon_method,
        'multipolygon_stats': multipolygon_stats
    }
    
    # Save metadata
    metadata_file = os.path.join(output_dir, f'et_data_{safe_name}_metadata.json')
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    # Print summary
    print(f"\n  ✓ Complete: {len(et_data_df):,} records")
    print(f"  Unique subcatchments: {metadata['num_subcatchments']}")
    print(f"  API Success rate: {fetch_stats['success_rate']:.1f}% ({fetch_stats['successful']}/{fetch_stats['total']})")
    if fetch_stats['failed'] > 0:
        print(f"  ⚠ {fetch_stats['failed']} subcatchments failed after retries")
    if missing_value_stats:
        print(f"  Missing value filtering: {missing_value_stats['subcatchments_with_missing']} subcatchments removed")
    if multipolygon_stats:
        print(f"  Multipolygon aggregation: {multipolygon_stats['method']}")
    if output_file:
        print(f"  Saved: {output_file} ({file_size_mb:.2f} MB)")
    
    return metadata

In [162]:
import yaml

# Load the watershed manifest
with open(SUBCATCHMENTS_MANIFEST, 'r') as f:
    manifest = yaml.safe_load(f)

watersheds = manifest['Subcatchments']
print(f"Loaded {len(watersheds)} watersheds from manifest")
print(f"\nFirst 5 watersheds:")
for w in watersheds[:5]:
    print(f"  - {w['name']}")

Loaded 395 watersheds from manifest

First 5 watersheds:
  - batch;;nasa-roses-2025;;wa-0
  - batch;;nasa-roses-2025;;wa-1
  - batch;;nasa-roses-2025;;wa-2
  - batch;;nasa-roses-2025;;wa-3
  - batch;;nasa-roses-2025;;wa-4


In [ ]:
# Configuration
START_DATE = '2020-01-01'
END_DATE = '2023-12-31'
DATASET = 'OPENET_CONUS'
VARIABLE = 'et_ensemble_mad'
MAX_CONCURRENT = 100  # Adjust based on API rate limits

# Watershed selection
# Option 1: Process all watersheds
PROCESS_ALL_WATERSHEDS = False

# Option 2: Process specific watersheds by name or index
# Examples:
#   SELECTED_WATERSHEDS = ['batch;;nasa-roses-2025;;wa-0', 'batch;;nasa-roses-2025;;wa-1']
#   SELECTED_WATERSHEDS = [0, 1, 2]  # Process first 3 watersheds
SELECTED_WATERSHEDS = [1, 2, 3]

print(f"Configuration:")
print(f"  Date range: {START_DATE} to {END_DATE}")
print(f"  Dataset: {DATASET}")
print(f"  Variable: {VARIABLE}")
print(f"  Max concurrent: {MAX_CONCURRENT}")
print(f"{'='*60}\n")

# Determine which watersheds to process
if PROCESS_ALL_WATERSHEDS:
    watersheds_to_process = watersheds
    print(f"Processing ALL {len(watersheds)} watersheds")
else:
    # Handle both name-based and index-based selection
    watersheds_to_process = []
    for selection in SELECTED_WATERSHEDS:
        if isinstance(selection, int):
            if 0 <= selection < len(watersheds):
                watersheds_to_process.append(watersheds[selection])
            else:
                print(f"⚠ Warning: Index {selection} out of range (0-{len(watersheds)-1})")
        elif isinstance(selection, str):
            matching = [w for w in watersheds if w['name'] == selection]
            if matching:
                watersheds_to_process.append(matching[0])
            else:
                print(f"⚠ Warning: Watershed '{selection}' not found in manifest")
    
    print(f"Processing {len(watersheds_to_process)} selected watershed(s):")
    for w in watersheds_to_process:
        print(f"  - {w['name']}")

print(f"\n{'='*60}")

# Process each watershed
all_metadata = []
for watershed in watersheds_to_process:
    metadata = await process_single_watershed(
        watershed_name=watershed['name'],
        geojson_url=watershed['url'],
        dataset=DATASET,
        variable=VARIABLE,
        start_date=START_DATE,
        end_date=END_DATE,
        max_concurrent=MAX_CONCURRENT
    )
    if metadata:
        all_metadata.append(metadata)

# Final summary
print(f"\n{'='*60}")
print(f"ALL WATERSHEDS COMPLETE")
print(f"{'='*60}")
print(f"Processed: {len(all_metadata)}/{len(watersheds_to_process)} watersheds")

if all_metadata:
    total_records = sum(m['num_records'] for m in all_metadata)
    total_subcatchments = sum(m['num_subcatchments'] for m in all_metadata)
    
    print(f"Total records: {total_records:,}")
    print(f"Total subcatchments: {total_subcatchments:,}")
    
    # Save summary metadata
    summary_file = 'data/processing_summary.json'
    summary = {
        'date_created': pd.Timestamp.now().isoformat(),
        'total_watersheds_processed': len(all_metadata),
        'total_records': int(total_records),
        'total_subcatchments': int(total_subcatchments),
        'date_range': {'start': START_DATE, 'end': END_DATE},
        'dataset': DATASET,
        'variable': VARIABLE,
        'watersheds': all_metadata
    }
    with open(summary_file, 'w') as f:
        json.dump(summary, f, indent=2)
    print(f"\nSummary saved: {summary_file}")

INFO:root:Loading subcatchments for batch;;nasa-roses-2025;;wa-1...


Configuration:
  Date range: 2020-01-01 to 2023-12-31
  Dataset: OPENET_CONUS
  Variable: et_ensemble_mad
  Max concurrent: 100

Processing 3 selected watershed(s):
  - batch;;nasa-roses-2025;;wa-1
  - batch;;nasa-roses-2025;;wa-2
  - batch;;nasa-roses-2025;;wa-3


Processing watershed: batch;;nasa-roses-2025;;wa-1


INFO:root:Loaded 12 subcatchment features (geometries)
INFO:root:Found 9 unique TopazIDs
INFO:root:Sample TopazIDs: [32, 33, 41, 42, 51]
INFO:root:Found 9 unique TopazIDs
INFO:root:Sample TopazIDs: [32, 33, 41, 42, 51]


Loaded 12 subcatchments
Starting concurrent fetch for 12 subcatchments...
Max concurrent requests: 100


Fetching ET data: 100%|██████████| 12/12 [00:33<00:00,  2.83s/it]
INFO:root:Loading subcatchments for batch;;nasa-roses-2025;;wa-2...

INFO:root:Loading subcatchments for batch;;nasa-roses-2025;;wa-2...


...

INFO:root:Loaded 21 subcatchment features (geometries)
INFO:root:Found 13 unique TopazIDs
INFO:root:Sample TopazIDs: [22, 23, 31, 32, 33]
INFO:root:Found 13 unique TopazIDs
INFO:root:Sample TopazIDs: [22, 23, 31, 32, 33]


Loaded 21 subcatchments
Starting concurrent fetch for 21 subcatchments...
Max concurrent requests: 100


Fetching ET data:  48%|████▊     | 10/21 [00:28<00:31,  2.88s/it]



CancelledError: 

ERROR:root:Exception during ET API request (after retries): RuntimeError: Session is closed
ERROR:root:Exception during ET API request (after retries): RuntimeError: Session is closed
ERROR:root:Exception during ET API request (after retries): RuntimeError: Session is closed
ERROR:root:Exception during ET API request (after retries): RuntimeError: Session is closed
ERROR:root:Exception during ET API request (after retries): RuntimeError: Session is closed
ERROR:root:Exception during ET API request (after retries): RuntimeError: Session is closed
ERROR:root:Exception during ET API request (after retries): RuntimeError: Session is closed
ERROR:root:Exception during ET API request (after retries): RuntimeError: Session is closed
ERROR:root:Exception during ET API request (after retries): RuntimeError: Session is closed
ERROR:root:Exception during ET API request (after retries): RuntimeError: Session is closed
ERROR:root:Exception during ET API request (after retries): RuntimeError: Sessio

In [ ]:
# Load data for a specific watershed
watershed_name = 'batch;;nasa-roses-2025;;wa-0'  # Change to your watershed name
safe_name = watershed_name.replace(';;', '_').replace('/', '_')
data_file = f'data/et_data_{safe_name}.parquet'
metadata_file = f'data/et_data_{safe_name}_metadata.json'

# Check if file exists
if os.path.exists(data_file):
    # Load the data
    df = pd.read_parquet(data_file)
    
    print(f"Watershed: {watershed_name}")
    print(f"="*60)
    
    # Display basic info
    print(f"\nDataset Summary:")
    print(f"  Total records: {len(df):,}")
    print(f"  Unique subcatchments: {df['topaz_id'].nunique()}")
    print(f"  Date range: {df['year'].min()}-{df['year'].max()}")
    
    # Sample data
    print(f"\nSample data:")
    print(df[['topaz_id', 'year', 'month', 'model', 'value']].head(10))
    
    # Summary statistics
    print(f"\nSummary Statistics:")
    print(df['value'].describe())
    
    # Load and display metadata
    if os.path.exists(metadata_file):
        with open(metadata_file, 'r') as f:
            metadata = json.load(f)
        print(f"\nMetadata:")
        print(f"  Created: {metadata['date_created']}")
        print(f"  Dataset: {metadata['dataset']}")
        print(f"  Variable: {metadata['variable']}")
else:
    print(f"No data found for watershed: {watershed_name}")
    print(f"Looking for: {data_file}")
    print(f"\nAvailable watershed files:")
    if os.path.exists('data'):
        parquet_files = [f for f in os.listdir('data') if f.endswith('.parquet') and f.startswith('et_data_')]
        for f in parquet_files:
            print(f"  - {f}")
    else:
        print("  (data directory does not exist yet)")

print(f"\n{'='*60}")
print(f"To load data for a different watershed:")
print(f"  1. Change the watershed_name variable above")
print(f"  2. Re-run this cell")
print(f"\nTo load the processing summary:")
print(f"  with open('data/processing_summary.json', 'r') as f:")
print(f"      summary = json.load(f)")

Watershed: batch;;nasa-roses-2025;;wa-0

Dataset Summary:
  Total records: 10,224
  Unique subcatchments: 213
  Date range: 2020-2023

Sample data:
   topaz_id  year  month            model     value
0        61  2020      1  et_ensemble_mad    3.9896
1        61  2020      2  et_ensemble_mad   15.7719
2        61  2020      3  et_ensemble_mad   34.0795
3        61  2020      4  et_ensemble_mad   66.8073
4        61  2020      5  et_ensemble_mad   94.5728
5        61  2020      6  et_ensemble_mad   95.0670
6        61  2020      7  et_ensemble_mad  116.2993
7        61  2020      8  et_ensemble_mad  106.3052
8        61  2020      9  et_ensemble_mad   69.5023
9        61  2020     10  et_ensemble_mad   27.6604

Summary Statistics:
count    10224.000000
mean        59.011291
std         46.984771
min          0.000000
25%         13.610937
50%         44.871000
75%        104.521325
max        167.000000
Name: value, dtype: float64


JSONDecodeError: Expecting value: line 45 column 31 (char 1066)

## Analyze Results

Load and analyze the processed watershed data.

## Random Scratch

In [ ]:
# Diagnostic: Check the et_data DataFrame
print(f"DataFrame shape: {et_data.shape}")
print(f"DataFrame columns: {et_data.columns.tolist()}")
print(f"DataFrame empty: {et_data.empty}")
print(f"\nFirst few rows:")
print(et_data.head())

if et_data.empty:
    print("\n⚠️  WARNING: No data was collected!")
    print("Possible reasons:")
    print("1. All API responses contained only NoData (-9999.0) values")
    print("2. API responses didn't contain 'data' key")
    print("3. Date range has no available data")
    print("\nTry running a single request manually to check the API response format.")

NameError: name 'et_data' is not defined

In [ ]:
# Sequential vs Concurrent Performance Comparison
async def compare_sequential_vs_concurrent(num_tests=10):
    """
    Compare sequential vs concurrent execution to measure real performance gains.
    """
    import time
    from tqdm.asyncio import tqdm
    
    sample_df = subcatchment_data.head(num_tests)
    
    print(f"{'='*70}")
    print(f"PERFORMANCE COMPARISON: Sequential vs Concurrent")
    print(f"{'='*70}")
    print(f"Testing with {num_tests} subcatchments...\n")
    
    # ========== SEQUENTIAL TEST ==========
    print("🐌 Running SEQUENTIAL test...")
    sequential_start = time.time()
    sequential_results = []
    
    async with aiohttp.ClientSession() as session:
        for idx, row in sample_df.iterrows():
            req_start = time.time()
            response = await fetch_et_timeseries(
                session=session,
                geometry=row['geometry'],
                dataset='OPENET_CONUS',
                variable='et_ensemble_mad',
                start_date='2023-01-01',
                end_date='2023-01-31',
                area_reducer='mean'
            )
            req_time = time.time() - req_start
            sequential_results.append((idx, req_time, response is not None))
            print(f"  Request {idx + 1}/{num_tests} completed in {req_time:.2f}s")
    
    sequential_time = time.time() - sequential_start
    sequential_request_times = [r[1] for r in sequential_results]
    sequential_success = sum(1 for r in sequential_results if r[2])
    
    print(f"\n✓ Sequential completed in {sequential_time:.2f}s")
    print(f"  Success rate: {sequential_success}/{num_tests}")
    print(f"  Avg per request: {sum(sequential_request_times)/len(sequential_request_times):.2f}s\n")
    
    # ========== CONCURRENT TEST ==========
    print("🚀 Running CONCURRENT test...")
    concurrent_start = time.time()
    concurrent_results = []
    
    async with aiohttp.ClientSession() as session:
        semaphore = asyncio.Semaphore(50)
        
        async def timed_fetch(idx, row):
            req_start = time.time()
            async with semaphore:
                response = await fetch_et_timeseries(
                    session=session,
                    geometry=row['geometry'],
                    dataset='OPENET_CONUS',
                    variable='et_ensemble_mad',
                    start_date='2023-01-01',
                    end_date='2023-01-31',
                    area_reducer='mean'
                )
            req_time = time.time() - req_start
            return idx, req_time, response is not None
        
        tasks = [
            asyncio.create_task(timed_fetch(idx, row))
            for idx, row in sample_df.iterrows()
        ]
        
        for coro in tqdm.as_completed(tasks, total=len(tasks), desc="  Processing"):
            result = await coro
            concurrent_results.append(result)
    
    concurrent_time = time.time() - concurrent_start
    concurrent_request_times = [r[1] for r in concurrent_results]
    concurrent_success = sum(1 for r in concurrent_results if r[2])
    
    print(f"\n✓ Concurrent completed in {concurrent_time:.2f}s")
    print(f"  Success rate: {concurrent_success}/{num_tests}")
    print(f"  Avg per request: {sum(concurrent_request_times)/len(concurrent_request_times):.2f}s\n")
    
    # ========== RESULTS SUMMARY ==========
    speedup = sequential_time / concurrent_time
    time_saved = sequential_time - concurrent_time
    time_saved_pct = (time_saved / sequential_time) * 100
    
    print(f"{'='*70}")
    print(f"RESULTS SUMMARY")
    print(f"{'='*70}")
    print(f"Sequential time:  {sequential_time:.2f}s")
    print(f"Concurrent time:  {concurrent_time:.2f}s")
    print(f"Time saved:       {time_saved:.2f}s ({time_saved_pct:.1f}%)")
    print(f"Speedup factor:   {speedup:.2f}x")
    print(f"\n{'='*70}")
    
    # Extrapolate to full dataset
    full_dataset_size = len(subcatchment_data)
    estimated_sequential = (sequential_time / num_tests) * full_dataset_size
    estimated_concurrent = (concurrent_time / num_tests) * full_dataset_size
    
    print(f"EXTRAPOLATION FOR {full_dataset_size} SUBCATCHMENTS:")
    print(f"{'='*70}")
    print(f"Estimated sequential time:  {estimated_sequential/60:.1f} minutes ({estimated_sequential/3600:.2f} hours)")
    print(f"Estimated concurrent time:  {estimated_concurrent/60:.1f} minutes ({estimated_concurrent/3600:.2f} hours)")
    print(f"Estimated time saved:       {(estimated_sequential - estimated_concurrent)/60:.1f} minutes")
    print(f"{'='*70}")
    
    if speedup > 5:
        print("🎉 Excellent! Concurrency provides significant performance gains!")
    elif speedup > 2:
        print("👍 Good! Concurrency is working well.")
    else:
        print("⚠️  Limited concurrency gains. API may be rate-limiting.")

# Run comparison test
await compare_sequential_vs_concurrent(num_tests=10)

### Understanding the Results

The comparison test shows **limited concurrency gains** (1.28x speedup). This indicates the ClimateEngine API is **rate-limiting concurrent requests**:

- **Sequential**: Each request takes ~2s (normal API response time)
- **Concurrent**: Each request takes ~8s (4x slower due to throttling)

**Why this happens:**
- The API detects multiple concurrent requests from the same source
- It throttles responses to prevent overwhelming the server
- This is common for public APIs

**Options to improve performance:**
1. **Reduce concurrency** - Try `max_concurrent=5` or `10` instead of `50`
2. **Add delays** - Space out requests with small delays
3. **Accept the limitation** - 1.28x is still faster than sequential
4. **Contact API provider** - Ask about rate limits and best practices

In [ ]:
# Test different concurrency levels to find optimal setting
async def test_optimal_concurrency(num_tests=10):
    """
    Test different concurrency levels to find the sweet spot.
    """
    import time
    from tqdm.asyncio import tqdm
    
    sample_df = subcatchment_data.head(num_tests)
    concurrency_levels = [1, 3, 5, 10, 20, 50]
    
    print(f"{'='*70}")
    print(f"TESTING OPTIMAL CONCURRENCY LEVEL")
    print(f"{'='*70}")
    print(f"Testing {num_tests} requests at different concurrency levels...\n")
    
    results = []
    
    for max_concurrent in concurrency_levels:
        print(f"Testing max_concurrent={max_concurrent}...", end=' ')
        start_time = time.time()
        
        async with aiohttp.ClientSession() as session:
            semaphore = asyncio.Semaphore(max_concurrent)
            
            async def fetch_one(idx, row):
                async with semaphore:
                    return await fetch_et_timeseries(
                        session=session,
                        geometry=row['geometry'],
                        dataset='OPENET_CONUS',
                        variable='et_ensemble_mad',
                        start_date='2023-01-01',
                        end_date='2023-01-31',
                        area_reducer='mean'
                    )
            
            tasks = [
                asyncio.create_task(fetch_one(idx, row))
                for idx, row in sample_df.iterrows()
            ]
            
            responses = await asyncio.gather(*tasks, return_exceptions=True)
        
        elapsed = time.time() - start_time
        success_count = sum(1 for r in responses if r is not None and not isinstance(r, Exception))
        avg_per_request = elapsed / num_tests
        
        results.append({
            'max_concurrent': max_concurrent,
            'total_time': elapsed,
            'avg_per_request': avg_per_request,
            'success_rate': success_count / num_tests
        })
        
        print(f"{elapsed:.2f}s (avg {avg_per_request:.2f}s/req, {success_count}/{num_tests} success)")
    
    # Find optimal
    print(f"\n{'='*70}")
    print(f"RESULTS:")
    print(f"{'='*70}")
    print(f"{'Level':<10} {'Total Time':<15} {'Avg/Request':<15} {'Success Rate'}")
    print(f"{'-'*70}")
    
    best_result = min(results, key=lambda x: x['total_time'])
    
    for r in results:
        marker = "  ⭐ BEST" if r == best_result else ""
        print(f"{r['max_concurrent']:<10} {r['total_time']:>6.2f}s{' '*7} "
              f"{r['avg_per_request']:>6.2f}s{' '*7} "
              f"{r['success_rate']*100:>5.1f}%{marker}")
    
    print(f"\n{'='*70}")
    print(f"RECOMMENDATION: Use max_concurrent={best_result['max_concurrent']}")
    print(f"Expected time for 555 subcatchments: {(best_result['avg_per_request'] * 555)/60:.1f} minutes")
    print(f"{'='*70}")

# Run optimization test
await test_optimal_concurrency(num_tests=10)

In [ ]:
# Definitive concurrency test with timing overlap analysis
async def prove_concurrency_works(num_tests=20):
    """
    Definitive test that records exact start/end times to prove requests overlap.
    If requests truly run concurrently, many should be IN PROGRESS at the same time.
    """
    import time
    
    sample_df = subcatchment_data.head(num_tests)
    
    print(f"{'='*70}")
    print(f"DEFINITIVE CONCURRENCY TEST")
    print(f"{'='*70}\n")
    
    # Test with high concurrency
    max_concurrent = 20
    print(f"Running {num_tests} requests with max_concurrent={max_concurrent}\n")
    
    request_events = []  # Track when each request starts and ends
    
    async with aiohttp.ClientSession() as session:
        semaphore = asyncio.Semaphore(max_concurrent)
        
        async def fetch_with_timing(idx, row):
            start_time = time.time()
            async with semaphore:
                print(f"[{time.time():.2f}] Request {idx+1} STARTED")
                response = await fetch_et_timeseries(
                    session=session,
                    geometry=row['geometry'],
                    dataset='OPENET_CONUS',
                    variable='et_ensemble_mad',
                    start_date='2023-01-01',
                    end_date='2023-01-31',
                    area_reducer='mean'
                )
                end_time = time.time()
                duration = end_time - start_time
                print(f"[{end_time:.2f}] Request {idx+1} FINISHED (took {duration:.2f}s)")
                return {
                    'id': idx + 1,
                    'start': start_time,
                    'end': end_time,
                    'duration': duration,
                    'success': response is not None
                }
        
        test_start = time.time()
        tasks = [
            asyncio.create_task(fetch_with_timing(idx, row))
            for idx, row in sample_df.iterrows()
        ]
        
        results = await asyncio.gather(*tasks)
        test_end = time.time()
    
    total_time = test_end - test_start
    
    # Analyze overlap
    print(f"\n{'='*70}")
    print(f"OVERLAP ANALYSIS")
    print(f"{'='*70}")
    
    # For each point in time, count how many requests were active
    time_points = []
    for r in results:
        time_points.append((r['start'], 1))   # +1 active request
        time_points.append((r['end'], -1))     # -1 active request
    
    time_points.sort()
    
    max_concurrent_actual = 0
    current_concurrent = 0
    
    for t, delta in time_points:
        current_concurrent += delta
        max_concurrent_actual = max(max_concurrent_actual, current_concurrent)
    
    avg_duration = sum(r['duration'] for r in results) / len(results)
    success_count = sum(1 for r in results if r['success'])
    
    print(f"Total time:                {total_time:.2f}s")
    print(f"Success rate:              {success_count}/{num_tests}")
    print(f"Avg request duration:      {avg_duration:.2f}s")
    print(f"Max concurrent (observed): {max_concurrent_actual}")
    print(f"Max concurrent (setting):  {max_concurrent}")
    
    # Calculate theoretical times
    theoretical_sequential = avg_duration * num_tests
    theoretical_concurrent = avg_duration  # If perfect concurrency
    
    print(f"\n{'='*70}")
    print(f"CONCURRENCY PROOF")
    print(f"{'='*70}")
    print(f"If SEQUENTIAL (1 at a time): Would take {theoretical_sequential:.2f}s")
    print(f"If CONCURRENT ({max_concurrent} at once): Should take ~{avg_duration:.2f}s")
    print(f"Actual time:                  {total_time:.2f}s")
    
    efficiency = (theoretical_sequential / total_time) / max_concurrent_actual
    
    print(f"\nActual max concurrent:        {max_concurrent_actual} requests")
    print(f"Concurrency efficiency:       {efficiency*100:.1f}%")
    
    print(f"\n{'='*70}")
    if max_concurrent_actual >= 10:
        print(f"✅ CONCURRENCY IS WORKING! {max_concurrent_actual} requests ran simultaneously.")
    elif max_concurrent_actual >= 3:
        print(f"⚠️  LIMITED CONCURRENCY. Only {max_concurrent_actual} requests ran simultaneously.")
        print(f"   API is likely limiting you to {max_concurrent_actual} concurrent connections.")
    else:
        print(f"❌ CONCURRENCY NOT WORKING. Only {max_concurrent_actual} requests ran simultaneously.")
        print(f"   This is essentially sequential execution.")
    print(f"{'='*70}")

await prove_concurrency_works(num_tests=20)